In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import warnings
warnings.simplefilter(action='ignore')

In [2]:
train = pd.read_csv('StudentPerformancePred.csv')
labels = pd.read_csv('train_labels.csv')

labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]) )
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )

train['session_id'] = train['session_id'].astype(str)
labels['session'] = labels['session'].astype(str)

In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
import warnings
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score 
from sklearn.metrics import accuracy_score
warnings.simplefilter(action='ignore')

train = pd.read_csv('StudentPerformancePred.csv')
labels = pd.read_csv('train_labels.csv')

labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]) )
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )

train['session_id'] = train['session_id'].astype(str)
labels['session'] = labels['session'].astype(str)

total_accuracy = 0

for q_no in range(1, 19):
    if q_no <= 3:
        select = train[train['level_group'] == '0-4']
    elif q_no <= 13:
        select = train[train['level_group'] == '5-12']
    else:
        select = train[train['level_group'] == '13-22']

    train_X, validate_X = train_test_split(select, test_size=0.2)
    train_y = labels.loc[(labels['session'].isin(train_X['session_id'])) & (labels['q'] == q_no)]
    validate_y = labels.loc[(labels['session'].isin(validate_X['session_id'])) & (labels['q'] == q_no)]

    train_X = train_X.drop(['session_id', 'level_group'], axis=1)
    train_y = train_y['correct']

    validate_X = validate_X.drop(['session_id', 'level_group'], axis=1)
    validate_y = validate_y['correct']

    model = xgb.XGBClassifier(eval_metric='error')
    model.fit(train_X, train_y)

    # y_pred = model.predict(validate_X)
    # print(cross_val_score(validate_y, y_pred, cv=5))

    question_accuracy = np.mean(cross_val_score(model, validate_X, validate_y, cv=5))
    total_accuracy += question_accuracy
    print("Question", q_no, "accuracy:", question_accuracy)

overall_accuracy = total_accuracy / 18
print("Overall accuracy:", overall_accuracy)


Question 1 accuracy: 0.6774901891915623
Question 2 accuracy: 0.980691788640401
Question 3 accuracy: 0.9363464842070188
Question 4 accuracy: 0.7865487793620666
Question 5 accuracy: 0.5327763180705748
Question 6 accuracy: 0.7475068276022
Question 7 accuracy: 0.6942465771929943
Question 8 accuracy: 0.555272394872938
Question 9 accuracy: 0.7029496592390461
Question 10 accuracy: 0.4977737401300903
Question 11 accuracy: 0.5593038885249001
Question 12 accuracy: 0.8533847570544385
Question 13 accuracy: 0.6889441251100409
Question 14 accuracy: 0.66581966124286
Question 15 accuracy: 0.5064655647941136
Question 16 accuracy: 0.6934002472121094
Question 17 accuracy: 0.6535079128138277
Question 18 accuracy: 0.9473796191852808
Overall accuracy: 0.704433807469248


In [4]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
warnings.simplefilter(action='ignore')

train = pd.read_csv('StudentPerformancePred.csv')
labels = pd.read_csv('train_labels.csv')

labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]) )
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )

train['session_id'] = train['session_id'].astype(str)
labels['session'] = labels['session'].astype(str)

total_accuracy = 0

for q_no in range(1, 19):
    if q_no <= 3:
        select = train[train['level_group'] == '0-4']
    elif q_no <= 13:
        select = train[train['level_group'] == '5-12']
    else:
        select = train[train['level_group'] == '13-22']

    train_X, validate_X = train_test_split(select, test_size=0.2)
    train_y = labels.loc[(labels['session'].isin(train_X['session_id'])) & (labels['q'] == q_no)]
    validate_y = labels.loc[(labels['session'].isin(validate_X['session_id'])) & (labels['q'] == q_no)]

    train_X = train_X.drop(['session_id', 'level_group'], axis=1)
    train_y = train_y['correct']

    validate_X = validate_X.drop(['session_id', 'level_group'], axis=1)
    validate_y = validate_y['correct']

    param_dist = {
        'n_estimators': randint(100, 500),
        'max_depth': randint(1, 50),
        'min_samples_split': randint(2, 20),
        'min_samples_leaf': randint(1, 20),
        'criterion': ['gini', 'entropy'],
    }

    base_model = RandomForestClassifier()

    random_search = RandomizedSearchCV(base_model, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1)

    random_search.fit(train_X, train_y)

    best_params = random_search.best_params_
    print(f"Best hyperparameters for question {q_no}:", best_params)

    best_model = RandomForestClassifier(**best_params)
    best_model.fit(train_X, train_y)

    question_accuracy = np.mean(cross_val_score(best_model, validate_X, validate_y, cv=5))
    total_accuracy += question_accuracy
    print("Question", q_no, "accuracy:", question_accuracy)

overall_accuracy = total_accuracy / 18
print("Overall accuracy:", overall_accuracy)


Best hyperparameters for question 1: {'criterion': 'entropy', 'max_depth': 42, 'min_samples_leaf': 6, 'min_samples_split': 10, 'n_estimators': 342}
Question 1 accuracy: 0.7237429444358139
Best hyperparameters for question 2: {'criterion': 'gini', 'max_depth': 33, 'min_samples_leaf': 4, 'min_samples_split': 9, 'n_estimators': 333}
Question 2 accuracy: 0.9828135800050883
Best hyperparameters for question 3: {'criterion': 'entropy', 'max_depth': 21, 'min_samples_leaf': 15, 'min_samples_split': 15, 'n_estimators': 278}
Question 3 accuracy: 0.9323149905550565
Best hyperparameters for question 4: {'criterion': 'entropy', 'max_depth': 38, 'min_samples_leaf': 13, 'min_samples_split': 13, 'n_estimators': 304}
Question 4 accuracy: 0.7984298203546976
Best hyperparameters for question 5: {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 6, 'min_samples_split': 15, 'n_estimators': 391}
Question 5 accuracy: 0.5533629177333035
Best hyperparameters for question 6: {'criterion': 'gini', 'max_de

In [1]:
import pandas as pd
from catboost import CatBoostClassifier
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
warnings.simplefilter(action='ignore')

# Assuming you have already read and preprocessed the data
train = pd.read_csv('CatBoostData.csv')
labels = pd.read_csv('train_labels.csv')

# Specify the categorical columns to be used by the CatBoost model
cat_features = ['event_name_mode', 'fqid_mode', 'room_fqid_mode', 'text_mode']

labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]) )
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )

train['session_id'] = train['session_id'].astype(str)
labels['session'] = labels['session'].astype(str)

total_accuracy = 0

# Loop through questions, create train & validation sets for each question
for q_no in range(1, 19):

    # Filter dataset based on current question
    if q_no <= 3:
        select = train[train['level_group'] == '0-4']
    elif q_no <= 13:
        select = train[train['level_group'] == '5-12']
    else:
        select = train[train['level_group'] == '13-22']

    # Create train & validation sets
    train_X, validate_X = train_test_split(select, test_size=0.2)
    train_y = labels.loc[(labels['session'].isin(train_X['session_id'])) & (labels['q'] == q_no)]
    validate_y = labels.loc[(labels['session'].isin(validate_X['session_id'])) & (labels['q'] == q_no)]

    train_X = train_X.drop(['session_id', 'level_group'], axis=1)
    train_y = train_y['correct']

    validate_X = validate_X.drop(['session_id', 'level_group'], axis=1)
    validate_y = validate_y['correct']

    # Train the CatBoost model
    model = CatBoostClassifier(cat_features=cat_features, iterations=500, learning_rate=0.1, depth=6, verbose=0)
    model.fit(train_X, train_y)

    # Calculate the accuracy using cross_val_score
    question_accuracy = np.mean(cross_val_score(model, validate_X, validate_y, cv=5))
    print("Question", q_no, "accuracy:", question_accuracy)
    total_accuracy += question_accuracy

overall_accuracy = total_accuracy / 18
print("Overall accuracy:", overall_accuracy)


Question 1 accuracy: 0.7133465269850703
Question 2 accuracy: 0.9770849234385448
Question 3 accuracy: 0.9352853633770344
Question 4 accuracy: 0.7927016140834352
Question 5 accuracy: 0.5085909585210502
Question 6 accuracy: 0.7697874381125422
Question 7 accuracy: 0.7114352486643116
Question 8 accuracy: 0.5730950821000871
Question 9 accuracy: 0.7156813080177327
Question 10 accuracy: 0.516230217965431
Question 11 accuracy: 0.6176515750203196
Question 12 accuracy: 0.8597492305579385
Question 13 accuracy: 0.7165256116698524
Question 14 accuracy: 0.6857632392441343
Question 15 accuracy: 0.5001010912906139
Question 16 accuracy: 0.7269229297111581
Question 17 accuracy: 0.6649688283091637
Question 18 accuracy: 0.9505618559370307
Overall accuracy: 0.718637946833636
